In [3]:
import openai 
from utils import call_chatgpt_api_all_chats, stream_process_chatgpt_response
from secret import gao_key as key 
from resources.generate_response import analyze_situation_rag, analyze_situation_rag_guidance
import concurrent.futures
import re
import time 
import json 
from benefits.eligibility_check import eligibility_check

openai.api_key = key

# Orchestration 2

## High-level idea: 
Have the baseline LLM generate an initial response (e.g., SMART goals, follow-up questions) along with relevant RAG-sourced materials, then call LLM modules with specified prompts to refine or modify the response as needed.

## Step 1: Generate initial response with baseline LLM

In [4]:
def call_llm(messages, model="gpt-4o-mini"):
    response = openai.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

In [5]:
def generate_smart_goals_and_questions(situation: str):
    system_prompt = (
        "You are a behavioral health specialist helping peer support workers create actionable recovery plans for clients. "
        "Given a client's situation, you will generate: "
        "1. Relevant and helpful SMART (Specific, Measurable, Achievable, Relevant, Time-bound) goals. "
        "2. Follow-up questions that a peer specialist can ask the client to better understand their needs, motivations, and potential barriers.\n\n"
        "Ensure your language is empathetic, person-centered, and strength-based."
    )

    user_prompt = f"Here is the client’s situation:\n{ situation.strip() }\n\nPlease generate SMART goals and follow-up questions."

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    return call_llm(messages)

In [9]:
# example_situation_1 = ("A 54-year-old man in Newark, NJ, is unhoused and recovering from (undiagnosed) alcohol use disorder. He has an extensive trauma history but has also never been formally diagnosed with PTSD.  He is formerly incarcerated, has no ID, and needs help securing housing, food, and legal support for applying for government benefits like SSI and Medicaid. ")
# example_situation_2 = ("A 28-year-old woman in Paterson, NJ, is living in temporary housing while working a part-time job and has a physical disability that doesn't prevent her from working entirely but makes it difficult. She has documented immigration status but struggles to balance employment demands with physical wellness and social engagement. She is uncertain enough about her sustained ability to earn enough from part-time work to make it that she is asking about disability benefits and housing subsidies. ")
# example_situation_3 = ("A trans man in Jersey City, NJ, aged 35, recently moved into permanent housing after recovering from substance use and trauma related to domestic violence. He is employed part-time, has ID, and seeks support with maintaining mental health and building community connections. He also wants to pursue a restraining order against his former partner.")
# example_situation_4 = ("A 60-year-old man in Camden, NJ, living with family and managing schizophrenia, is unemployed and struggling to apply for SSI and food stamps. He is a non-drug user with ID but needs assistance navigating benefits (determining eligibility, applying). He also needs to find a primary care provider and get help with transportation to appointments.  Several of his physical health conditions are linked to his prescribed psychiatric medications (i.e. diabetes, metabolic syndrome).")
example_situation_5 = ("A 72-year-old retired man in Morristown, NJ, living in permanent housing and dually enrolled (Medicaid/Medicare), seeks support for maintaining his physical wellness through better diet and exercise. He is a non-drug user with ID and no incarceration history but struggles with isolation, spending most of his days alone in his room, and wants to build social connections.")

response = generate_smart_goals_and_questions(example_situation)
print(response)

### SMART Goals

1. **Dietary Improvement Goal**: 
   - **Specific**: The client will prepare a balanced meal at home five times a week, primarily incorporating fruits, vegetables, whole grains, and lean proteins.
   - **Measurable**: Track the number of home-cooked meals prepared each week using a simple chart.
   - **Achievable**: Given his interest in improving health and potential access to cooking resources, this goal is realistic.
   - **Relevant**: This goal aligns with his desire to enhance physical wellness through better nutrition.
   - **Time-bound**: The client will achieve this goal over the next month.

2. **Exercise Goal**: 
   - **Specific**: The client will engage in a moderate-intensity exercise (like walking or chair exercises) for at least 30 minutes, four times a week.
   - **Measurable**: Use a log to record the type of exercise and duration on each day.
   - **Achievable**: Given the client's age and physical capacity, this is a feasible goal with proper support.

## Step 2: RAG for resources recommendation 
At this stage, we continue to rely on our RAG framework to retrieve relevant resources. Directly prompting the LLM to generate resources is not effective, as our goal is to provide materials that have been validated by CSPNJ and to minimize hallucinations. Therefore, we avoid using the baseline LLM for resource retrieval and instead depend on the RAG system to ensure accuracy and reliability.

In [45]:
def extract_resource_keywords(situation: str):
    system_prompt = (
        "You are an assistant for a behavioral health agency that helps clients access wellness-related resources. "
        "Given a client’s situation, extract a list short keyword phrases that describe the types of help or resources they may need. "
        "Each phrase should end in the word 'help' (e.g., 'housing stability help', 'social connection help', 'diet help'). "
        "Take into account the 8 Dimensions of Wellness: emotional, physical, occupational, social, spiritual, intellectual, environmental, and financial. "
        "Focus on the client’s challenges, unmet needs, and wellness goals. Make sure the keywords are short (3–6 words) and phrased as resource needs."
    )

    user_prompt = f"Client Situation:\n{ situation.strip() }\n\nList the keywords for resources they need."

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    return call_llm(messages)


In [46]:
keywords = extract_resource_keywords(example_situation)
print("Resource Keywords:\n", keywords)

Resource Keywords:
 1. diet help  
2. exercise help  
3. social connection help  
4. physical wellness help  
5. group activity help  
6. community engagement help  
7. nutritional guidance help  
8. fitness program help  
9. companionship help  
10. wellness support help


In [47]:
goal_prompt = open("mental_health/prompts/mental_health_prompt.txt").read()
question_prompt = open("mental_health/prompts/question_prompts.txt").read()
benefit_prompt = open("benefits/prompts/uncertain_prompt_wellness.txt").read()
resource_prompt = open("mental_health/prompts/resource_prompt.txt").read()

In [57]:
rag_input = f"""
User Situation:
{example_situation}
Resource keywords:
{keywords}
"""
rag_input

'\nUser Situation:\nA 72-year-old retired man in Morristown, NJ, living in permanent housing and dually enrolled (Medicaid/Medicare), seeks support for maintaining his physical wellness through better diet and exercise. He is a non-drug user with ID and no incarceration history but struggles with isolation, spending most of his days alone in his room, and wants to build social connections.\nResource keywords:\n1. diet help  \n2. exercise help  \n3. social connection help  \n4. physical wellness help  \n5. group activity help  \n6. community engagement help  \n7. nutritional guidance help  \n8. fitness program help  \n9. companionship help  \n10. wellness support help\n'

In [58]:
retrieved_resources = analyze_situation_rag(rag_input, k=10)
retrieved_resources

"Friends of Fairview Give Back, URL: https://www.facebook.com/p/Friends-of-Fairview-Give-Back-100070266467554/, Phone: nan, Description: Friends of Fairview Give Back is a community-driven initiative that focuses on enhancing the mental health and well-being of individuals in need. It provides supportive services such as stress management workshops, peer support groups, and mental health awareness campaigns. The program fosters a nurturing environment through community events and fundraising efforts aimed at improving access to mental health resources. By promoting collaboration among local organizations, Friends of Fairview Give Back empowers individuals to seek help, share experiences, and build supportive networks for better mental health outcomes.\nReach Out Addiction and Recovery, URL: https://988lifeline.org/chat, Phone: nan, Description: Reach Out Addiction and Recovery offers a comprehensive online pre-chat survey to facilitate access to the 988 Lifeline, ensuring individuals s

In [51]:
def resource_selection_prompt(situation, keywords, retrieved_resources):
    return f"""
You are an expert resource recommendation assistant. Given the user's specific situation, conceptual keyword guidance, and a list of potential resources retrieved from a knowledge base, your task is to carefully select and recommend ONLY the most relevant and helpful resources. 

Consider the original situation and the conceptual keyword guidance carefully to ensure selected resources specifically match the user's needs and context.

Do not make anything up. Only select resources from the list that is given to you.

## Original User Situation:
{situation}

## Conceptual Keyword Guidance for Selecting Resources:
{keywords}

## Retrieved Resources:
{retrieved_resources}

## Your task:
- Carefully evaluate the retrieved resources in relation to the user's situation and the provided conceptual guidance.
- Select and recommend ONLY the resources that BEST match the user's situation.
- Clearly present your recommendations in a structured, easy-to-read format.
- For each selected resource, briefly explain why it is specifically relevant.

## Selected Most Relevant Resources (clearly listed and explained):
"""

def select_relevant_resources(situation, meta_reasoning_guidance, retrieved_resources):
    prompt = resource_selection_prompt(situation, meta_reasoning_guidance, retrieved_resources)
    messages = [
        {"role": "system", "content": "You are an expert assistant selecting highly relevant resources tailored to the user's situation."},
        {"role": "user", "content": prompt}
    ]

    selected_resources_response = call_llm(messages)
    
    return selected_resources_response

In [52]:
def format_selected_resources(raw_text):
    formatted_text = raw_text.replace("\\n", "\n")
    formatted_text = re.sub(r'\*\*', '', formatted_text)
    formatted_text = re.sub(r'###+', '', formatted_text)
    formatted_text = re.sub(r'\s*(\d+)\.\s*(.+?):\s*\n', r'\n\1. **\2**\n', formatted_text)
    formatted_text = re.sub(r'–\s*URL\s*:', '- **URL:**', formatted_text, flags=re.I)
    formatted_text = re.sub(r'–\s*Phone\s*:', '- **Phone:**', formatted_text, flags=re.I)
    formatted_text = re.sub(r'–\s*Relevance\s*:', '- **Relevance:**', formatted_text, flags=re.I)
    formatted_text = re.sub(r'\n+', '\n', formatted_text).strip()
    return formatted_text

In [59]:
selected_resources = select_relevant_resources(
        example_situation,
        keywords,
        retrieved_resources)
resource_output = format_selected_resources(selected_resources)
print(resource_output)

## Selected Most Relevant Resources
1. Friends of Fairview Give Back  
   - URL: [Friends of Fairview Give Back](https://www.facebook.com/p/Friends-of-Fairview-Give-Back-100070266467554/)  
   - Relevance: This resource focuses on enhancing mental health and well-being, providing supportive services such as peer support groups and community events. For the user, who struggles with isolation, engaging in community events can foster social connections and combat feelings of loneliness.
2. Soshimsa Zen Center  
   - URL: [Soshimsa Zen Center](https://www.soshimsa.org/)  
   - Relevance: The Zen Center promotes holistic well-being through meditation, health exercises, and community outreach. The user can participate in mindful practices and connect with others in a nurturing environment, aiding his physical wellness and social interaction.
3. Our Lady of Mount Carmel  
   - URL: [Our Lady of Mount Carmel](https://www.olmcridgewood.com/)  
   - Relevance: This parish community offers social

## Step 3: Call individual modules to modify section of the responses as needed. 

In [61]:
def refine_response_with_dual_prompts(
    original_response: str,
    smart_goal_prompt: str,
    followup_prompt: str,
    situation: str
):
    system_prompt = (
        "You are a behavioral health recovery planning assistant helping peer specialists support clients. "
        "You are given a client situation and an original response (which includes SMART goals and follow-up questions). "
        "Your task is to revise the SMART goals and follow-up questions **only as needed** based on the two separate refinement prompts provided. "
        "Maintain the SMART goal format (Specific, Measurable, Achievable, Relevant, Time-bound), and use strength-based, empathetic, person-centered language. "
        "If any section already meets the refinement prompt, preserve it as-is."
    )

    user_prompt = (
        f"Client Situation:\n{ situation.strip() }\n\n"
        f"SMART Goal Refinement Instructions:\n{ smart_goal_prompt.strip() }\n\n"
        f"Follow-Up Question Refinement Instructions:\n{ followup_prompt.strip() }\n\n"
        f"Original Response:\n{ original_response.strip() }"
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    return call_llm(messages)


In [64]:
print(response)

### SMART Goals

1. **Dietary Improvement Goal**: 
   - **Specific**: The client will prepare a balanced meal at home five times a week, primarily incorporating fruits, vegetables, whole grains, and lean proteins.
   - **Measurable**: Track the number of home-cooked meals prepared each week using a simple chart.
   - **Achievable**: Given his interest in improving health and potential access to cooking resources, this goal is realistic.
   - **Relevant**: This goal aligns with his desire to enhance physical wellness through better nutrition.
   - **Time-bound**: The client will achieve this goal over the next month.

2. **Exercise Goal**: 
   - **Specific**: The client will engage in a moderate-intensity exercise (like walking or chair exercises) for at least 30 minutes, four times a week.
   - **Measurable**: Use a log to record the type of exercise and duration on each day.
   - **Achievable**: Given the client's age and physical capacity, this is a feasible goal with proper support.

In [63]:
smart_goal_prompt = open("mental_health/prompts/mental_health_prompt.txt").read()
followup_prompt = open("mental_health/prompts/question_prompts.txt").read()

refined_response = refine_response_with_dual_prompts(
    response,
    smart_goal_prompt,
    followup_prompt,
    example_situation
)

print(refined_response)


### SMART Goals

1. **Dietary Improvement Goal**: 
   - **Specific**: The client will prepare a balanced meal at home five times a week, focusing on a variety of fruits, vegetables, whole grains, and lean proteins.
   - **Measurable**: Track the number of home-cooked meals prepared each week using a simple chart or journal.
   - **Achievable**: Considering his interest in health and potential access to cooking resources, this goal is realistic.
   - **Relevant**: This goal supports his aspiration to enhance physical wellness through better nutrition.
   - **Time-bound**: The client will reach this goal within the next month.

2. **Exercise Goal**: 
   - **Specific**: The client will engage in moderate-intensity exercises (like walking or chair exercises) for at least 30 minutes, four times a week.
   - **Measurable**: Use a log to document the type of exercise and duration for each session.
   - **Achievable**: Given the client's age and physical capacity, this is a feasible goal with 

## Step 4: Generate final & holistic response to present

In [67]:
def generate_holistic_response(
    refined_response: str,
    retrieved_resources: str,
    situation: str
):
    system_prompt = (
        "You are a behavioral health recovery planning assistant creating a holistic support summary for a peer specialist to use in their session. "
        "You are given: (1) the client situation, (2) a refined list of SMART goals and follow-up questions, and (3) relevant resources retrieved by the system.\n\n"
        "Your task is to write a holistic, easy-to-follow response that:\n"
        "- Introduces the client’s goals and strengths in an affirming tone,\n"
        "- List the SMART goals in natural language,\n"
        "- Weaves in the retrieved resources (along with contact information such as links and phone numbers if given) as suggestions for each goal,\n"
        "- Recommends follow-up questions to support engagement and motivation,\n"
        "- Is warm, person-centered, and recovery-oriented."
    )

    user_prompt = (
        f"Client Situation:\n{ situation.strip() }\n\n"
        f"Refined SMART Goals and Follow-Up Questions:\n{ refined_response.strip() }\n\n"
        f"Retrieved Resources:\n{ retrieved_resources.strip() }\n\n"
        f"Please generate a holistic response as described."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    return call_llm(messages)


In [68]:
holistic_response = generate_holistic_response(
    refined_response=refined_response,
    retrieved_resources=resource_output,
    situation=example_situation
)

print(holistic_response)

Hello! It's great to be with you today, and I want to start by acknowledging the wonderful goals you've set for yourself as you navigate this journey toward improved physical wellness and greater social connections. Your commitment to enhancing your diet and engaging in regular exercise, while also seeking to build relationships, shows a remarkable strength and desire for overall well-being. Let’s dive into your SMART goals and explore some resources to support you along the way.

### Your SMART Goals:

1. **Dietary Improvement Goal**: You will prepare a balanced meal at home five times a week, focusing on a diverse selection of fruits, vegetables, whole grains, and lean proteins. To keep track of your accomplishments, you can use a simple chart or journal over the next month.

   - **Suggested Resource**: Consider connecting with **Friends of Fairview Give Back**, which provides supportive services such as cooking classes or meal planning workshops. Engaging in these programs can equi

# Pros and Cons 

## Original Approach (module output --> summarize output --> final output)

#### Pros
- fewer LLM calls --> faster overall
- simple and easy to maintain 
#### Cons
- less tailored response in comparison to Orch. v1 (All outputs from Orch v1 (SMART goals, Resources, Followup questions) are more detailed and tailored based on the responses from 5 scenarios)
- Bruteforce workflow, not as sophisticated  

## Orchestration v1. (reason first --> use meta reasoning as context --> agentic RAG --> final output)

#### Pros
- More tailored and detailed recommendation overall due to the reasoning step (more context and clearer guidance for LLM)
- A bit more reasonable resource recommendation thanks to the agentic workflow for RAG 
#### Cons
- Slower due to the reasoning step

## Orchestration v2. (baseline LLM response --> call modules to modify as needed --> NLU + RAG --> final output)

#### Pros
- Maintains the generalalizability (is that a word?) of the original LLM
#### Cons
- Not much different from the original approach 